This notebook goal is to plot a figure using no custom function before we get to design and write useful ones

# Imports

In [2]:
# Third Parties
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np 

# Native

# Custom 
# /

# Classes
from plotly.graph_objs._figure import Figure as goFigure
# Functions
from plotly.subplots import make_subplots
from itertools import product

# Open file and prepare for plotting

In [3]:
articles = pd.read_csv(
    '../../data/2025-01-07-2024-10-29-quinquadef4-neat-abstract-bert.csv'
)

revues = pd.read_csv(
    '../../data/Classification revues - Feuille 1.csv'
)

/var/folders/xs/d90v1vkn16db_7z493h6kwt40000gn/T/ipykernel_30339/3563665539.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  articles = pd.read_csv(


In [4]:
articles.columns

Index(['iddef', 'titre', 'soustitre', 'auteurs', 'annee', 'revue', 'num_titre',
       'num_num', 'abstract', 'url_art', 'auteurs_fam', 'auteurs_prenom',
       'source', 'n_auteurs', 'femme', 'annee5', 'majo_femmes', 'sexe3',
       'tag_race', 'tag_inclusif', 'bert_pronoms', 'bert_quanti', 'bert_quali',
       'bert_genre', 'bert_genre_stat', 'bert_classe_large',
       'bert_classe_stricte', 'bert_genre_pas_stat', 'bert_monostricte',
       'bert_interstricte_2plus', 'bert_interstricte_3',
       'bert_interstricte_GC', 'bert_interstricte_GR', 'bert_interstricte_CR',
       'bert_interstricte_cl', 'bert_interstricte_cl2', 'bert_monolarge',
       'bert_interlarge_2plus', 'bert_interlarge_3', 'bert_interlarge_GC',
       'bert_interlarge_GR', 'bert_interlarge_CR', 'bert_interlarge_cl',
       'bert_interlarge_cl2'],
      dtype='object')

In [5]:
revues.columns

Index(['revue', 'Dominante', 'Aréale', 'Anthropo', 'Demo', 'Eco', 'Genre',
       'Géographie', 'Histoire', 'SIC', 'Sociologie', 'SciencePo'],
      dtype='object')

In [6]:
# 'Économie' and 'Economie' exist in the revue.Dominante column
# We only want 'Economie' :
revues.loc[:,"Dominante"] = revues.loc[
                            :,
                            'Dominante'
                            ].replace("Économie","Economie")

In [7]:
CAT = [
    'Aréale'        ,
    'Anthropologie' ,
    'Démographie'   ,
    'Economie'      ,
    'Genre'         ,
    'Géographie'    ,
    'Histoire'      ,
    'SIC'           ,
    'Sociologie'    ,
    'Science politique' 
]


In [29]:
unknown_revues = []
unknown_categories = []

found_categorie : list[str] = []

for i in range(len(articles)): 
    # retrieve the name of the revue
    revue_i : str = articles.loc[i,'revue']

    # find the category
    try : 
        found_categorie_i : str = revues.loc[
                                    revues.revue == revue_i,
                                    'Dominante'].item()
    except:
        unknown_revues.append(revue_i)
        found_categorie_i = "NaN"

    if found_categorie_i in CAT : 
        cat_label : str = found_categorie_i
    if found_categorie_i == "NaN":
        cat_label : str = "NaN"
    else : 
        if found_categorie_i != 'Autre interdisciplinaire' : 
            unknown_categories.append(found_categorie)
            
        cat_label : str = 'Autre interdisciplinaire'
    found_categorie.append(found_categorie_i)


In [30]:
unknown_revues

['Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Commentaire',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations',
 'Éducation & formations']

In [31]:
set(found_categorie)

{'Anthropologie',
 'Aréale',
 'Autre interdisciplinaire',
 'Démographie',
 'Economie',
 'Genre',
 'Géographie',
 'Histoire',
 'NaN',
 'SIC',
 'Science politique',
 'Sociologie'}

In [32]:
articles['Discipline'] = found_categorie

In [33]:
articles.loc[articles.Discipline == "NaN",['revue','Discipline']]

,revue,Discipline
5916,Commentaire,NaN
5917,Commentaire,NaN
5918,Commentaire,NaN
5919,Commentaire,NaN
5920,Commentaire,NaN
5921,Commentaire,NaN
5922,Commentaire,NaN
5923,Commentaire,NaN
5924,Commentaire,NaN
5925,Commentaire,NaN


In [34]:
# Create usable df
df = articles.loc[
    articles.Discipline != "NaN",
    ["annee", "bert_genre", "Discipline"]
]
df = df.sort_values('annee')
df.index = np.arange(0,len(df))
df.head()

,annee,bert_genre,Discipline
0,2001,False,Histoire
1,2001,False,Science politique
2,2001,False,Science politique
3,2001,False,Science politique
4,2001,False,Science politique


In [35]:
set(df.Discipline)

{'Anthropologie',
 'Aréale',
 'Autre interdisciplinaire',
 'Démographie',
 'Economie',
 'Genre',
 'Géographie',
 'Histoire',
 'SIC',
 'Science politique',
 'Sociologie'}

In [36]:
# create the df to plot for all the categories
# NOTE mind the nan in bert_genre
# >>> no 'nan' found in the 'bert_genre' column

# Do not plot the year 2023
dfPlot = {
    'annee' : list(set(df.annee) - set([2023]))
                        # already sorted because of previous manipulations
}


for y,cat in product(dfPlot['annee'], set(df.Discipline)):

    if cat not in dfPlot:
        dfPlot[cat] = []
    dfPlot[cat].append(
        df.loc[
            (df.annee == y) & (df.Discipline == cat),
            "bert_genre"
        ].mean() * 100
    )

# Add the "Toutes" groupe
dfPlot['Toutes'] = []
for y in dfPlot['annee']:

    dfPlot['Toutes'].append(
        df.loc[
            (df.annee == y),
            "bert_genre"
        ].mean() * 100
    )

# Add the "Toutes Sauf Genre" groupe
dfPlot['Toutes sauf Genre'] = []
for y in dfPlot['annee']:

    dfPlot['Toutes sauf Genre'].append(
        df.loc[
            (df.annee == y) & (df.Discipline != "Genre"),
            "bert_genre"
        ].mean() * 100
    )

In [37]:
dfPlot = pd.DataFrame(dfPlot)
dfPlot

,annee,Aréale,Genre,SIC,Economie,Anthropologie,Démographie,Géographie,Autre interdisciplinaire,Sociologie,Science politique,Histoire,Toutes,Toutes sauf Genre
0,2001,3.571429,92.592593,3.448276,0.000000,8.750000,25.000000,0.000000,0.588235,4.242424,1.000000,2.614379,7.601185,2.815433
1,2002,4.109589,91.228070,0.000000,1.639344,7.317073,24.000000,0.000000,8.108108,3.184713,4.678363,11.351351,10.254163,5.996310
2,2003,2.739726,83.333333,7.586207,0.000000,7.594937,26.086957,0.000000,6.719368,8.500000,3.141361,1.704545,9.318996,5.291005
3,2004,7.377049,95.522388,2.469136,7.446809,5.681818,15.000000,0.000000,3.448276,6.091371,2.427184,3.191489,8.311518,4.312115
4,2005,1.923077,89.855072,5.384615,0.000000,17.241379,29.411765,0.000000,9.489051,3.125000,2.515723,4.232804,9.650259,5.898305
5,2006,1.307190,93.827160,1.388889,0.000000,8.602151,23.809524,0.000000,5.894309,5.418719,7.647059,10.695187,9.512894,5.408654
6,2007,5.035971,90.625000,1.574803,1.111111,4.629630,30.555556,2.702703,6.488550,15.087719,4.184100,3.781513,9.205867,6.481965
7,2008,2.688172,91.044776,2.352941,0.000000,3.773585,15.625000,2.127660,4.273504,6.287425,1.968504,3.396226,6.450072,3.652517
8,2009,5.327869,83.146067,7.065217,1.666667,6.593407,11.538462,2.564103,12.213740,5.389222,5.084746,5.415162,10.212569,7.084337
9,2010,4.291845,95.180723,2.762431,2.054795,15.740741,26.000000,2.222222,6.151142,5.523256,1.107011,1.433692,8.185362,4.941599


### Running average

In [38]:
windowsize = 5

def running_average(vec : np.ndarray, windowsize : int = 3) -> np.ndarray :
    window = np.ones(windowsize) / windowsize
    # NOTE check avec EOLLION
    return np.convolve(vec,window,mode = 'valid')

dfPlotRA = pd.DataFrame({
    **{'annee' : dfPlot.loc[
                            range(windowsize - 1,len(dfPlot)),
                            'annee'
                            ]},
    **{key : running_average(   dfPlot.loc[:,key],
                                windowsize
                            ) for key in dfPlot.keys() if key != 'annee'}
})

In [39]:
dfPlotRA

,annee,Aréale,Genre,SIC,Economie,Anthropologie,Démographie,Géographie,Autre interdisciplinaire,Sociologie,Science politique,Histoire,Toutes,Toutes sauf Genre
4,2005,3.944174,90.506291,3.777647,1.817231,9.317041,23.899744,0.000000,5.670608,5.028702,2.752526,4.618914,9.027224,4.862634
5,2006,3.491326,90.753205,3.365769,1.817231,9.287472,23.661649,0.000000,6.731822,5.263961,4.081938,6.235076,9.409566,5.381278
6,2007,3.676603,90.632591,3.680730,1.711584,8.749983,24.972760,0.540541,6.407911,7.644562,3.983086,4.721108,9.199907,5.478409
7,2008,3.666292,92.174879,2.634077,1.711584,7.985713,22.880369,0.966072,5.918738,7.202047,3.748514,5.059444,8.626122,5.150711
8,2009,3.256456,89.699615,3.553293,0.555556,8.168030,22.188061,1.478893,7.671831,7.061617,4.280026,5.504179,9.006332,5.705156
9,2010,3.730209,90.764745,3.028856,0.966514,7.867902,21.505708,1.923337,7.004249,7.541268,3.998284,4.944356,8.713353,5.513815
10,2011,3.735438,89.446122,3.713645,1.283975,11.418790,22.626156,1.923337,6.533022,7.710425,2.935798,3.154969,8.486657,5.451136
11,2012,3.510853,89.821122,3.730176,1.211006,11.960754,24.082613,2.335178,7.329175,8.194207,2.382666,3.754599,8.877280,5.805791
12,2013,3.620992,87.908463,3.644204,1.778382,14.602264,26.513168,2.818737,7.539563,8.361223,3.270104,4.156434,9.352770,6.287694
13,2014,2.945662,89.025728,3.659732,4.322747,16.047810,29.538809,2.305916,6.583302,8.478184,2.646598,3.578817,9.050856,6.116471


# Plotting

In [92]:
toute_discipline = list(set(dfPlot.columns) - set(['annee']))
fig = px.line(
    dfPlotRA,
    x = 'annee', y = toute_discipline,
    # color = 'Discipline',
    # labels = {
    #         'y' : 'Dominante de la revue',
    #     }
)

In [93]:
# Default parameters
fig

### Customisation

In [94]:
# https://plotly.com/python-api-reference/generated/plotly.subplots.make_subplots.html

# Facetting
fig = make_subplots(
    rows=11, cols=1,
    specs=[[{}]] * 11,
    row_heights= [0.5] + [0.05] * 10,
    subplot_titles=(
       "","Science Politique","Sociologie","Anthropologie",
       "Histoire", "Démographie", "Économie","SIC", "Géographie", "Aréales", 
       "Autre interdisciplinaire"),

       shared_xaxes = 'all',
       print_grid = True,

       x_title = 'Année de publication',
       y_title = ''
)

# Setting title and axis names
fig = fig.update_layout(dict(
    title = dict(
            text = ("Évolution de la proportion d'articles publiés invoquant le "
                    "thème du genre (def extensive et statistique)"),
             automargin = True,  # TODO Check that again
             pad = dict(t = 10, b = 10 , r = 10, l = 10)),
    width = 1150, height = 1400,
    plot_bgcolor = 'white',
    showlegend = False
))

This is the format of your plot grid:
[ (1,1) x,y      ]
[ (2,1) x2,y2    ]
[ (3,1) x3,y3    ]
[ (4,1) x4,y4    ]
[ (5,1) x5,y5    ]
[ (6,1) x6,y6    ]
[ (7,1) x7,y7    ]
[ (8,1) x8,y8    ]
[ (9,1) x9,y9    ]
[ (10,1) x10,y10 ]
[ (11,1) x11,y11 ]



In [95]:
all_categories = [
    'Science politique',
    'Sociologie'	,
    'Anthropologie'	,
    'Histoire'	,
    'Autre interdisciplinaire'	,
    'Démographie',
    'Economie',
    'SIC'	,
    'Géographie'	,
    'Aréale']


def add_traces_to_subplot(fig : goFigure, df : pd.DataFrame,
                          x : str, columns : list[str],
                          row : int, col : int, **kwargs) -> None:
    '''takes a dataframe and addit to the figure. 
    the kwargs can contain : 
        {
        # will be applied to the traces one at a time
         'local' : {
                        [kwargs, kwargs, ]
                    }, 
        # will be applied to all the traces
         'global' : kwargs 
        }
    all customisation must be in the kwargs
    '''
    
    for i, column  in zip( range(len(columns)), columns):
        fig.add_trace(
            go.Scatter(
                x = df[x], y = df[column],
                mode = 'lines'
            ), row = row, col = col
        )
        if 'global' in kwargs:
            fig.data[-1].update(kwargs['global'])
        if 'local' in kwargs: 
            print(kwargs['local'][i])
            fig.data[-1].update(**kwargs['local'][i])

## Create figure 1

In [96]:
grey = 'rgba(200,200,200,0.5)'
# Add all the categories

kwargs = {
    'global' : {
        'line' : {
            'color' : grey,
            'width' : 1}
    }
}

add_traces_to_subplot(fig,dfPlot,
                      x = "annee",columns=all_categories,
                      row = 1, col = 1, **kwargs) # All categories except gender studies
add_traces_to_subplot(fig, dfPlot,
                      x = "annee", columns=["Genre"],
                      row = 1, col = 1, **kwargs)

# Add the colourful traces
kwargs = {
    'local' : [
        {
            'name' : "Moyenne de toutes les revues",
            'line' : {'color' : 'red'}
        },
        {
            'name' :  ("Moyennes de toutes les revues qui ne sont pas"
                       " des revues spécialisées dans les études de genre"),
            'line' : {'color' : 'green'}
        },
        {
            'name' : "Revues spécialisées dans les études de genre",
            'line' : {'color' : 'black', 'width' : 40}
        }
    ]
}
add_traces_to_subplot(  fig, dfPlotRA,
                        x = "annee",
                        columns = ["Toutes", "Toutes sauf Genre", "Genre"],
                        row = 1, col = 1)


In [97]:
tick_grid_colour = 'rgb(50,50,50)'

fig = fig.update_layout(
    xaxis = dict(
        showline = True,
        linewidth = 1,
        linecolor = tick_grid_colour,

        ticks = 'outside',
        showticklabels = True,
        tickangle = 0,
        tickwidth = 1,
        tickfont = dict(),
        tickvals = [2000,2005,2010,2015,2020],
        
        showgrid = False
    ),
    yaxis = dict(
        showline = False,

        ticks = "outside",
        showticklabels = True,
        tickangle = 0,
        tickfont = dict(),
        tickvals = [10,50,92],
        ticksuffix = "%",
        tickwidth = 1,

        showgrid = True,
        gridcolor = tick_grid_colour,
        gridwidth = 1,

        autorange = True,
        # range = [1, 100],
    )
)

## Save Image

In [98]:
fig.write_image('fig_test.png')

In [99]:
fig.data[-4]

Scatter({
    'line': {'color': 'rgba(200,200,200,0.5)', 'width': 1},
    'mode': 'lines',
    'x': array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
                2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]),
    'xaxis': 'x',
    'y': array([92.59259259, 91.22807018, 83.33333333, 95.52238806, 89.85507246,
                93.82716049, 90.625     , 91.04477612, 83.14606742, 95.18072289,
                87.23404255, 92.5       , 81.48148148, 88.73239437, 93.93939394,
                96.        , 91.54929577, 93.67088608, 93.05555556, 95.        ,
                95.37037037, 91.30434783]),
    'yaxis': 'y'
})

In [100]:
# FIXME Gender studies et pas Genre